In [32]:
import pandas as pd 
import numpy as np

import os 

from tqdm import tqdm

import imageio
import cv2

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D
from tensorflow.keras.models import Model

import random



In [33]:
print('Number of infected cell image: ',len(os.listdir('./Dataset/Parasitized/')))

Number of infected cell image:  13779


In [34]:
print('Number of un-infected cell_images: ',len(os.listdir('./Dataset/Uninfected/')))

Number of un-infected cell_images:  13779


# splitting image dataset into train, test

## checking extension of each file is png or not

In [35]:
unifected_file_lst = os.listdir('./Dataset/Uninfected/')
infected_file_lst = os.listdir('./Dataset/Parasitized/')

unifected_file_lst

['C100P61ThinF_IMG_20150918_144104_cell_128.png',
 'C100P61ThinF_IMG_20150918_144104_cell_131.png',
 'C100P61ThinF_IMG_20150918_144104_cell_144.png',
 'C100P61ThinF_IMG_20150918_144104_cell_21.png',
 'C100P61ThinF_IMG_20150918_144104_cell_25.png',
 'C100P61ThinF_IMG_20150918_144104_cell_34.png',
 'C100P61ThinF_IMG_20150918_144104_cell_48.png',
 'C100P61ThinF_IMG_20150918_144104_cell_65.png',
 'C100P61ThinF_IMG_20150918_144348_cell_108.png',
 'C100P61ThinF_IMG_20150918_144348_cell_120.png',
 'C100P61ThinF_IMG_20150918_144348_cell_125.png',
 'C100P61ThinF_IMG_20150918_144348_cell_131.png',
 'C100P61ThinF_IMG_20150918_144348_cell_2.png',
 'C100P61ThinF_IMG_20150918_144348_cell_25.png',
 'C100P61ThinF_IMG_20150918_144348_cell_40.png',
 'C100P61ThinF_IMG_20150918_144348_cell_71.png',
 'C100P61ThinF_IMG_20150918_144823_cell_126.png',
 'C100P61ThinF_IMG_20150918_144823_cell_127.png',
 'C100P61ThinF_IMG_20150918_144823_cell_139.png',
 'C100P61ThinF_IMG_20150918_144823_cell_142.png',
 'C100P61T

In [36]:
for i in unifected_file_lst:
    
    temp = os.path.splitext(i)[1]
    
    if temp != '.png':
        print(i) 

for i in infected_file_lst :
    
    temp = os.path.splitext(i)[1]
    
    if temp != '.png':
        print(i)

## checking size of each image

In [37]:
image_size = set()


for i in tqdm(unifected_file_lst):
    
    image_size.add(imageio.imread('./Dataset/Uninfected/'+i).shape)
    
image_size



  0%|                                                                                        | 0/13779 [00:00<?, ?it/s]C:\Users\91951\AppData\Local\Temp/ipykernel_33388/4129995760.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image_size.add(imageio.imread('./Dataset/Uninfected/'+i).shape)
100%|██████████████████████████████████████████████████████████████████████████| 13779/13779 [00:12<00:00, 1145.22it/s]


{(76, 70, 3),
 (148, 184, 3),
 (97, 121, 3),
 (139, 127, 3),
 (133, 169, 3),
 (82, 106, 3),
 (196, 226, 3),
 (76, 109, 3),
 (169, 178, 3),
 (112, 157, 3),
 (130, 112, 3),
 (115, 175, 3),
 (106, 115, 3),
 (127, 166, 3),
 (70, 145, 3),
 (109, 100, 3),
 (160, 112, 3),
 (136, 166, 3),
 (106, 154, 3),
 (85, 103, 3),
 (151, 175, 3),
 (142, 202, 3),
 (139, 100, 3),
 (73, 103, 3),
 (169, 118, 3),
 (172, 169, 3),
 (88, 79, 3),
 (115, 148, 3),
 (112, 97, 3),
 (130, 85, 3),
 (166, 94, 3),
 (163, 154, 3),
 (148, 214, 3),
 (139, 157, 3),
 (199, 163, 3),
 (151, 148, 3),
 (142, 142, 3),
 (184, 148, 3),
 (94, 127, 3),
 (91, 76, 3),
 (196, 139, 3),
 (109, 130, 3),
 (121, 121, 3),
 (163, 127, 3),
 (97, 124, 3),
 (157, 130, 3),
 (199, 136, 3),
 (100, 109, 3),
 (142, 115, 3),
 (196, 196, 3),
 (145, 130, 3),
 (172, 199, 3),
 (187, 136, 3),
 (88, 109, 3),
 (130, 115, 3),
 (181, 139, 3),
 (133, 178, 3),
 (223, 145, 3),
 (124, 118, 3),
 (127, 169, 3),
 (166, 124, 3),
 (187, 175, 3),
 (109, 103, 3),
 (130, 154

--so we have many sized image so I will **resize it into 120*120 images**

## making train test folder and in the folder i am making Parasitized and Unifected folder

In [38]:

destination = './Data_transfer_learning/'


directory_lst = [destination +'train/' +  'Parasitized', destination + 'train/'+'Uninfected', 
                 destination + 'test/' +  'Parasitized', destination + 'test/' + 'Uninfected']

directory_lst


['./Data_transfer_learning/train/Parasitized',
 './Data_transfer_learning/train/Uninfected',
 './Data_transfer_learning/test/Parasitized',
 './Data_transfer_learning/test/Uninfected']

In [39]:
if not os.path.exists(destination): os.mkdir(destination)

In [40]:
# creating test and train directory
if not os.path.exists(destination + 'train'): os.mkdir(destination + 'train')
if not os.path.exists(destination + 'test'): os.mkdir(destination + 'test')

In [41]:
# creating directory for each class in test and train respectevly
for i in directory_lst:
    
    if os.path.exists(i):
        True        
    else:
        os.mkdir(i)

## splitting data and storing in respective folder

In [42]:
# sampling images from parasitized class for train and test
# In train we are keeping 8k images and rest for test

par_train = random.sample(infected_file_lst, 8000)
par_test =  list(set(infected_file_lst) - set(par_train))



# sampling images from uninfected class for train and test
# In train we are keeping 8k images and rest for test

uninfected_train = random.sample(unifected_file_lst, 8000)
unifected_test = list(set(unifected_file_lst) - set(uninfected_train))

In [43]:
image_resize = (224,224)

# train folder parasited class folder

In [44]:
# copying images from dataset to respective train and test and their class folder

for image_name in tqdm(par_train):
    
    if image_name.endswith('png'):
        
        # reading image Dataset -> Parasitized
        image = imageio.imread('./Dataset/Parasitized/'+ image_name)
        
        # resizing image
        resized_image = cv2.resize(image, image_resize, interpolation=cv2.INTER_AREA)
        
        # saving image tp Data -> train -> Parasitized
        imageio.imsave('./Data_transfer_learning/train/Parasitized/' + image_name, resized_image)

  0%|                                                                                         | 0/8000 [00:00<?, ?it/s]C:\Users\91951\AppData\Local\Temp/ipykernel_33388/3650917174.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('./Dataset/Parasitized/'+ image_name)
100%|██████████████████████████████████████████████████████████████████████████████| 8000/8000 [12:27<00:00, 10.70it/s]


In [45]:
len(os.listdir(path='./Data/train/Parasitized/'))

8000

# test folder parasited class folder

In [46]:
for image_name in tqdm(par_test):
    
    if image_name.endswith('png'):
        
        # reading image from dataset->Parasitized
        image = imageio.imread('./Dataset/Parasitized/'+ image_name)
        
        # resizing image with dimension 128*128
        resized_image = cv2.resize(image, (128,128), interpolation=cv2.INTER_AREA)
        
        # saving image to Data_transfer_learning -> test -> Parasitized
        imageio.imsave('./Data_transfer_learning/test/Parasitized/' + image_name, resized_image)

  0%|                                                                                         | 0/5779 [00:00<?, ?it/s]C:\Users\91951\AppData\Local\Temp/ipykernel_33388/3543430410.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('./Dataset/Parasitized/'+ image_name)
100%|██████████████████████████████████████████████████████████████████████████████| 5779/5779 [02:37<00:00, 36.69it/s]


In [47]:
len(os.listdir('./Data_transfer_learning/test/Parasitized/'))

5779

# train folder unifected class folder

In [48]:
for image_name in tqdm(uninfected_train):
    
    if image_name.endswith('png'):
        
        # reading image from dataset -> Uninfected
        image = imageio.imread('./Dataset/Uninfected/' + image_name)
        
        #resizing image with dimension 128*128
        resized_image = cv2.resize(image, (128,128), interpolation=cv2.INTER_AREA)
        
        # saving image to Data_transfer_learning -> train -> Uninfected
        imageio.imsave('./Data_transfer_learning/train/Uninfected/'+image_name, resized_image)
        
        

  0%|                                                                                         | 0/8000 [00:00<?, ?it/s]C:\Users\91951\AppData\Local\Temp/ipykernel_33388/1817383756.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('./Dataset/Uninfected/' + image_name)
100%|██████████████████████████████████████████████████████████████████████████████| 8000/8000 [04:04<00:00, 32.77it/s]


In [49]:
len(os.listdir('./Data/train/Uninfected/'))

8000

# test folder unifected class folder

In [50]:
for image_name in tqdm(unifected_test):
    
    if image_name.endswith('png'):
        
        # reading image from dataset -> uninfected
        image = imageio.imread('./Dataset/Uninfected/' + image_name)
        
        # resizing image with dimension 128*128
        resized_image = cv2.resize(image, (128,128), interpolation=cv2.INTER_AREA)
        
        # saving image to Data_transfer_learning -> test -> uninfected
        imageio.imsave('./Data_transfer_learning/test/Uninfected/'+image_name, resized_image)
        

  0%|                                                                                         | 0/5779 [00:00<?, ?it/s]C:\Users\91951\AppData\Local\Temp/ipykernel_33388/876434708.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('./Dataset/Uninfected/' + image_name)
100%|██████████████████████████████████████████████████████████████████████████████| 5779/5779 [02:57<00:00, 32.52it/s]


In [51]:
len(os.listdir('./Data_transfer_learning/test/Uninfected/'))

5779